In [2]:
from pathlib import Path

import numpy as np
import rasterio
from rasterio.crs import CRS
from rasterio.warp import calculate_default_transform, reproject

In [3]:
output_crs = CRS.from_epsg(32647)
resolution = (0.0714906612329378, 0.0714906612329378)


In [12]:
def run(input_raster, output_raster):
    src = rasterio.open(input_raster)
    # 创建一个rasterio.crs.CRS对象来表示目标坐标系
    # 使用rasterio.warp.calculate_default_transform函数进行转换
    dst_transform, dst_width, dst_height = calculate_default_transform(src.crs, output_crs, src.width, src.height,
                                                                       *src.bounds,resolution=resolution)


    profile = src.meta.copy()
    profile.update({
        'crs': output_crs,
        'transform': dst_transform,
        'width': dst_width,
        'height': dst_height,
        'nodata': 0,
        'compress': 'deflate'
    })

    dst_img = np.zeros((dst_height, dst_width), dtype=profile['dtype'])
    # 重投影
    reproject(source=src.read(1),
              destination=dst_img,
              src_crs=src.crs,
              src_transform=src.transform,
              dst_transform=dst_transform,
              dst_crs=output_crs,
              num_threads=4)

    with rasterio.open(output_raster, 'w', **profile) as dst:
        dst.write(dst_img, 1)


# 设置目录路径和文件后缀
dir_path = Path(r'D:\UAV_DATA_NEW\deprecated\3_predict_result')
output_path = Path(r'D:\UAV_DATA_NEW\output\1_UTM47N')
# 循环处理每个文件
for input_raster in dir_path.glob(f'*.tif'):
    # 获取文件名和后缀
    file_name = input_raster.stem[0:6]
    output_raster = output_path / f'{file_name}_UTM47N.tif'
    print(input_raster.name)
    run(input_raster, output_raster)

061301_result.tif
061302_result.tif
061303_result.tif
061306_result.tif
061308_result.tif


In [24]:
def run(input_raster, output_raster):
    src = rasterio.open(input_raster)
    # 创建一个rasterio.crs.CRS对象来表示目标坐标系
    output_crs = CRS.from_epsg(32647)
    # 使用rasterio.warp.calculate_default_transform函数进行转换
    dst_transform, dst_width, dst_height = calculate_default_transform(src.crs, output_crs, src.width, src.height,
                                                                       *src.bounds)

    profile = src.meta.copy()
    profile.update({
        'crs': output_crs,
        'transform': dst_transform,
        'width': dst_width,
        'height': dst_height,
        'nodata': 0
    })

    origin_img = src.read([1, 2, 3])
    r, g, b = origin_img
    mask = (r == 255) & (g == 255) & (b == 255)

    origin_img[0][mask] = 0
    origin_img[1][mask] = 0
    origin_img[2][mask] = 0

    src_img = np.zeros((3, dst_height, dst_width), dtype=profile['dtype'])
    # 重投影
    reproject(source=origin_img,
              destination=src_img,
              src_crs=src.crs,
              src_transform=src.transform,
              dst_transform=dst_transform,
              dst_crs=output_crs,
              num_threads=4)

    with rasterio.open(output_raster, 'w', **profile) as dst:
        dst.write(src_img, [1, 2, 3])


# 设置目录路径和文件后缀
dir_path = Path(r'D:\UAV_DATA_NEW\0_raw_img')
output_path = Path(r'D:\UAV_DATA_NEW\0_raw_img_UTM47N')
# 循环处理每个文件
for input_raster in dir_path.glob(f'*.tif'):
    # 获取文件名
    file_name = input_raster.stem[6:12]
    output_raster = output_path / f'{file_name}_UTM47N.tif'
    print(output_raster)
    run(input_raster, output_raster)

D:\UAV_DATA_NEW\0_raw_img_UTM47N\061301_UTM47N.tif
D:\UAV_DATA_NEW\0_raw_img_UTM47N\061302_UTM47N.tif
D:\UAV_DATA_NEW\0_raw_img_UTM47N\061303_UTM47N.tif
D:\UAV_DATA_NEW\0_raw_img_UTM47N\061306_UTM47N.tif
D:\UAV_DATA_NEW\0_raw_img_UTM47N\061308_UTM47N.tif
